Some important points: \
Speedup = T(1) / T(p) -> p is the number of compute units used. \
Efficiency = T(1) / (p * T(p)). \
if computation time is $\alpha$ and communication time is $\beta$, and computation-communication time is given by $\gamma$, the efficieny drops with decrease in $\gamma$. more the $\gamma$, more the efficiency and speedup and also with more compute units p, there is an increase in efficiency till a global maximum after which the communication time starts dominating and reduces efficiency.

Distributed memory systems: \
In some parallel algorithms like above, each PE has access to its own local memory and if it needs to communicate with other PEs, it has to be explicitly done through an interconnection network. Remote data access needs to be facilitated through message passing over interconnection network. \
The interconnection network is an important architectural factor of a distributed memory system. \
Standard network protocols (such as Infiniband or Ethernet) are often used to implement the communication tasks. \
Network topology : determines the scalability of the architecuture for many applications. \
Prominent examples of distributed memory systems are compute clusters and network-on-chip (NOC) architectures. \
The message passing interface (MPI) is arguably the most popular language for parallel programming on distributed memory systems. \
Data exchanges between two processes can be implemented using (versions of) MPI_Send and MPI_Recv commands while data communication between groups of processes can be implemented by collective communication functions such as MPI_Bcast, MPI_Reduce, MPI_Gather, or MPI_Scatter. \
Data partitioning i.e distribution of data between the processes is a key issue in distributed memory systems. \
Partitioned Global Address Space (PGAS) is another popular approach to develop programs for distributed memory systems. \
The PGAS model is the basis of UPC++ .



Shared Memory systems : \
All CPUs (or cores) can access a common memory space through a shared bus or crossbar switch. \
Prominent examples of such systems are modern multi-core CPU-based workstations in which all cores share the same main memory. \
In addition to the shared main memory each core typically also contains a smaller local memory (e.g. Level 1 cache) in order to reduce expensive accesses to main memory (known as the von Neumann bottleneck). \
Cache coherence : n order to guarantee correctness, values stored in (writable) local caches must be coherent with the values stored in shared memory. \
Modern multi-core systems support cache coherence and are often also referred to as cache coherent non-uniform access architectures (ccNUMA) \
Parallelism is typically created by starting threads running concurrently on the system. Exchange of data is usually implemented by threads reading from and writing to shared memory locations. \
race conditions should be avoided. \
A race condition can occur when two threads access a shared variable simultaneously (without any locking or synchronization), which could lead to unexpected results. \
A number of programming techniques (such as mutexes, condition variables, atomics), which can be used to avoid race conditions. \
A program typically starts with one process running a single thread. This master thread creates a number of slave threads which later join the master thread in order to terminate. Each thread can define its own local variables but has also access to shared variables. Thread creation is much more lightweight and faster compared to process creation. Therefore threads are often dynamically created and terminated during program execution. \
OpenMP is another approach to multi-threaded programming (Chapter 6) based on semiautomatic parallelization. \
They simplify parallelization through APIs which operate through pragmas. Pragmas are prep-processor directives to compiler which can be used to generate multi-threaded code. \
Thus, when parallelizing a sequential code with OpenMP, a programmer often only needs to annotate the code with the suitable pragmas. Nevertheless, achieving a highly efficient and scalable implementation can still require in-depth knowledge. \

The utilized number of threads in a program can range from a small number (e.g., using one or two threads per core on a multi-core CPU) to thousands or even millions. This type of massive multi- threading is used on modern accelerator architectures




Considerations when designing parallel algorithms : \
Partitioning : The given project can be parallelize in different ways among the processing elements [cores] , task parallelism, model parallelism, data parallelism. \
Communication : the partition scheme depends on the amount and types of communication required between processes or threads. \
Synchronization : In order for parallel computations to work properly, the threads / processes need to be synchronized. \
Load balancing : The total work needs to be equally distributed among available threads or processes to not overburden any one thread/process. \


Finding parallelism in loop carried data dependency is difficult but not impossible, with careful study of data in han, it can be achieved with some clever tricks. Consider a for loop: \
for(int i=0;i<N; i++) A[i] = A[i-1] + A[i] \
let the array be : [6,3,5,8,1,8,0,2,4,3,5,2,1,7,4,1,2,3,5,1] of 20 elements.
let the array be divided into 4 cores with 5 elements each. \
perform the same computations on each of the cores will give: \
PE 1: [6,9,14,22,23] \
PE 2: [8,8,10,14,17] \
PE 3: [5,7,8,15,19] \
PE 4 : [1,3,6,11,12] \

These four PE's sums need to be synced. \
the last element of PE 1, can be taken and added with all elements in PE2, So, elements in PE 2 become : \
PE 2: [31,31,33,37,40] \
at the same time, the last element of PE 3 can be added to PE4 : \
PE 4 : [20,2,25,30,31].

The final step is to add PE2's last element  to PE 3 and PE 4: \
PE 3 : [45,47,48,55,59]
PE 4 : [60,62,65,70,71]

In [ ]:
my_list= [6,3,5,8,1,8,0,2,4,3,5,2,1,7,4,1,2,3,5,1]
new_list = []
new_list.append(my_list[0])
for i in range(1,len(my_list)):
    new_list.append(my_list[i]+new_list[i-1])
print(new_list)


[6, 9, 14, 22, 23, 31, 31, 33, 37, 40, 45, 47, 48, 55, 59, 60, 62, 65, 70, 71]


the above python code matches with our analysis, the difference being we are able to data parallelize this with some additional transfers required in between

Choice of partitioning strategy is crucial for parallel algorithm design.\ Data parallelism distributes the data across different processors or cores
which can then operate on their assigned data. \
Some data parallel
algorithms are even embarrassingly parallel and can operate independently on their assigned data; e.g.,
in an image classification task different images can be assigned to different processors which can then
classify each image independently in parallel. \
For the implementation of a data parallel algorithm you sometimes need to perform synchronization
between processes or threads. \
Task parallelism : \
Consider a ternary classifier, where each image passed through three binary classifiers and then the results are merged to get the result. \
Data parallelism would have this entire model on each core / PE, and different images are put through these cores giving out results. \
Task parallelism would mean having these three classifiers on three different cores / PE -> P0,P1,P2 and then merging is done on p0 for each image. \
To scale towards large number of processors, task parallelism and data parallism will be combined together. \

when multiple PEs process data, one of the operations in task parallelism could be expensive and could take more time to learn. for example : human classification. This might limit the achievable speedup because other two PEs are remaining idle. Load balancing makes sure that speedup is not reduced. \
In data parallel approach, the input image dataset is divided into batches and the images are fed batch by batch into the PE. Once a Process has completed its batch, a dynamic scheduler assigns a new batch to the idle PE.

to train neural networks, the images are fed to massively parallel GPUs which process apply matrix operations on these images and churn out outputs. Some neural networks are complex and their size (weights) can exceed the main memory of the GPU. So , the usual data-parallel approach does not work. The model itself is broken down into chunks and fed to individual computing components of the GPU. The weights are equally distributed among GPUs and each GPU does the matrix operation associated with that weight alone. the vector produced by each of the GPU after each layer need to be aligned before proceeding with the next layer.

High Performance Computers [HPC] : \
Most of them use neo-heteregeneous architecture containing millions of cores along with CUDA-based GPU accelerators which churn out billions of FLOPS/sec. \
They also employ different levels of parallelism : \
Node level parallelism : requires implementation of algorithms for distributed memory model using MPI .\
Intra-node parallelization : requires libraries which implement parallelism with shared memory [OpenMP] , [multithreading]. \
Accelerator-level parallelization : offloads some of the computations to dedicated accelerators like GPUs for parallization.

Additional Exercises: \
1.Analyze the speedup and the efficiency of the parallel summation algorithm presented in Section 1.1 using n = 2048 numbers assuming that each PE can add two numbers in one millisecond and each PE can send m numbers in 2 + m/1024 milliseconds to another PE. Vary the number of PEs from 1 to 1024 using powers of 2. \
Soln : n = 2048.\
let us consider the simpler case, where number of PE is 1. [the base case]
T(1) = n - 1 = 2047 ms. \
Speedup = T(1) / T(n) = T(1) / T(1) = 1 \
p = 2 ,[number of PEs = 2] \
the numbers can be split up into two PEs each. to transfer 2048/2 = 1024 numbers to PE1 from PE0, the time taken is 2 + 1024/1024 = 3 ms. \
The computation time is 1024 - 1 = 1023 ms in both PEs. to bring back the sum of 1024 elements to PE0, the time taken in 2 + 1/1024 = 2.0009765625 ms. \
To sum the two computed sums, the time taken is 1 ms.\
Total time taken = 3 + 2.001 + 1023 + 1 = 1029.001 ms. \
 Speedup = T(1) / T(2) = 2047 / 1029.001 = 1.989, efficiency = T(1) / (2*T(2)) = 0.994. \

 p = 2^2 = 4 : \
 PE 0 sends half its inputs to PE 1 in 2 + 1024/1024 = 3 ms. PE0 and then half of its current inputs to PE2 and PE1 can send half of its inputs to PE3 in 2 + 512/1024 = 2.5 ms. \
 Computation of 512-1 additions in 511 ms . PE1 sends sum to PE0 and PE3 send sum to PE 2 in 2+1/1024 ~ 2 ms. One addition in PE0 and PE 2 = 1 ms. Sending of data in PE 2 to PE 0 is ~ 2 ms and final addition is of time 1 ms. \
 total sum = 3 + 2.5 + 511 + 2+ 1+ 2+ 1 = 522.5 ms. \
 Speedup = T(1)/T(4) = 2047 / 522.5 = 3.917, efficiency = 0.9794. \

 p = 2^3 = 8 : \
 PE0 send half its inputs to PE 1 -> 2 + 1024/1024 = 3 ms. PE 0 and PE 1 send half of existing inputs to PE 2 and PE 3 in 2 + 512/1024 = 2.5 ms. PEs 0 , 1, 2, 3 send half of existing inputs to PE4, PE5, PE6, PE7 in 2 + 256/1024 = 2.25 ms. Addition time : 256-1 = 255 ms. Sum from PE7 can be transferred to PE3, from PE6 to PE 2, PE5 to PE1 and PE4 to PE 0 in 2 + 1/2024 = 2.0001 ms. the four addition operations can be done in 1 ms . \
 Sum from PE4 can be taken to PE3 can be taken to PE1 and PE 2 to PE 0 in 2.0001 ms. 1 ms for addition and one final movement to PE0 from PE 1 in 2.0001 ms and one final addition in 1 ms. \
 total time = 3 + 2.5 + 2.25 + 255 + 2.0001 + 1 + 2.0001 + 1 + 2.0001 + 1 = 271.7503 ms. speedup = 7.53265037794, efficieny = 7.53265037794/8 = 0.94158. \

 General trend : \
 Distribution of inputs among p cores : \
 let p = 2^m, then distribution of inputs to 2^m cores takes $2 \times m + \sum _{i=1}^{m} 1/m$. \
 time for addition : \
 $T_{add} = \frac{n}{p} - 1$ \
 Time for recollection : \
 $T_{recollection} $= (2.0001 + 1) * m = 3.0001m ~ 3m$. \

 Now, for n = 2048, p = 1024, m = 10. \
 Distribution time = 2 * 10 + (1 + 1/2 + 1/3 + 1/4 + 1/5 + 1/6 + 1/7 + ... 1/10) = 20 + 2.9289 = 22.9289. \
 Addition time = 2048/1024 - 1 = 1 ms \
 Collection time = 3 * 10 = 30 ms. \
Total time = 22.93 + 30 + 1 = 53.93 ms. \
Speedup = 2047 / 53.98 = 37.956, efficieny = 0.0370 -> due to communication time, there is a drop in efficieny with 1024 cores.


2. Consider the parallel prefix computation algorithm presented in Fig. 1.9. Describe how this parallel algorithm works in general on a shared memory machine using an input array of size n = $2^k$ and n/4 cores. How high is the achieved speedup? \
Soln : it is a shared memory system. On a single core, the prefix computation would take n - 1 time units  = $2^k$ - 1 time units. \
With n/4 cores, 4 numbers can be assigned to each core, meaning the time taken would be 4 - 1 time-units. Now the sums need to be synchronized. \
You will have n/4 numbers local sub-array last numbers which will be stored in another array. Prefix computation will be done in these numbers. this can be done in parallel to in $\log _2 {n/4}$ = k-2 time units. \
the remaining operation is to add these last numbers of local sub-arrays to next sub-array. that can be done in a vectorized way as wall thereby parallelizing the operation to be done in 1 time unit. \
Total time will thus be : 3 + k - 2 + 3 [if sequential or 1 if vectorized] = k + 4 units [assuming sequential addition] .
T(1) = 2^k - 1. \
Speedup = $\frac{2^k-1}{k+4}$

3. Histogram problem : \
The computation of a histogram is a frequent operation in image processing. The histogram simply counts the number of occurrences of each tonal value in the given image. Consider a 2-dimensional input gray-scale image I of size n × n.

        for (i=0; i<n; i++)
           for (j=0; j<n; j++)
               histogram[I[i,j]]++

Discuss the advantages and disadvantages of the two following partitioning strategies for computing an image histogram in parallel:
a. The histogram slots are partitioned between processors.
b. The input image is partitioned between processors. \

Soln : Baiscally we have an input array I of size n x n. Also output array ,histogram of size 256 [assuming 8 bit grayscale image wit value between 0-255]. if we sequentially compute the operations, we need to go through the entire n x n, update that index of histrogram which is given by the intensity of the image.
Case a. histogram slots are partitioned between processors : \
Advantages : \
Reduced conflicts : By assigning a portion of histrogram slots to be updated to different processors, we can prevent conflicts during write operations. The write operation is independent. \
Load balancing : for a image with a uniform distribution of intensity, every processor will be equally utilized. there is good load balancing in most scenarios \
Scalaibility : can be scaled to higher number of processors because each processor handles the assigned subset of histogram slots. \
Disadvantages : \
Global synchronization : The master core has to wait till all cores have done their job in filling the histogram values and combine them to form the final completed global histogram. \
Memory usage : each core has to have its dedicated share of histogram slots potentially leading to memory usage issues if the histogram is large. \
Imbalance in usage of cores : During cases where there is uneven tonal intensity distributions in images, there is imbalance in core usage.

Case b . The input image is partitioned between processors \
Advantages : \
Cache utilization : Potentially better usage of cache since chunk of image stored in contiguous blocks can be used with reduced memory access latency. \
Local processing : Each image block is independent of one another and the cores can read them and process the subsequent updation independently.
No Global sychronization  :there is just one single buffer of histgrams updated in a core and there is one final synchronization required.

Disadvantages : \
Load imbalancing : some patches of image may involve more computations than other patches potentially slowing down certain cores. For example, the edges may not have very uniform dostribution whereas center pixels may have distributed tonacy.  \
Write conflicts : Same index of output histogram slots may have to be filled by multiple cores handling different portion of image and thus we need to enforce locks for dat to prevent data race conditions. \
Complexity in combining results : There are multiple checks to be done so that same data is no improperly updated by different cores, there is some difficulty in combining the tonacy of each individual index in histogram

5. Write down all the prime numbers between 1 and 1000 on the board. At each step, you are allowed
to erase two numbers on the board (say, x and y) and in place of the two erased numbers write the number x + y + x · y. Repeat this process over and over until only a single number remains (call it Q). Over all possible combination of numbers, what is the smallest value of Q? Assume we have already precomputed the n prime numbers between 1 and 1000. Moreover, we use a third- party library for arbitrary long integers1 and thus do not need to worry about potential overflow of integers.
(i) Prove that the operation x ⊙ y := x + y + x · y is commutative and associative.
(ii) Using the result of (i), how can we efficiently parallelize this algorithm?
(iii) Investigate the runtime of the algorithm on p processor. Discuss the result. \


In [35]:
#python program to list all prime numbers between 1 and 1000.

def sieve_of_eratosthenes(max_num):
    """Return a list of all prime numbers up to max_num."""
    is_prime = [True] * (max_num + 1)  # Initialize a boolean list
    is_prime[0] = is_prime[1] = False  # 0 and 1 are not prime numbers

    for start in range(2, int(max_num**0.5) + 1):
        if is_prime[start]:
            for multiple in range(start*start, max_num + 1, start):
                is_prime[multiple] = False

    return [num for num, prime in enumerate(is_prime) if prime]

# Define the range
max_number = 1000

# Get all prime numbers up to max_number
primes = sieve_of_eratosthenes(max_number)

# Print the list of prime numbers
print(f"Prime numbers between 1 and {max_number}:")
print(primes)

Prime numbers between 1 and 1000:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]


In [36]:
len(primes)

168

there are 168 primes between 1 and 1000 and they have been listed above and arranged in ascending order. let us first erase the numbers pairwise in sequence and replace them with x + y + x.y as required in the problem.

In [22]:
new_primes1 = primes.copy()


In [23]:
len(new_primes1)

25

In [24]:
while(1):
  if len(new_primes1) == 1:
    break
  a = new_primes1[0]
  b = new_primes1[1]
  c = a + b + a*b # also a prime
  new_primes1.pop(0)
  new_primes1.pop(0)
  new_primes1.insert(0,c)

print(new_primes1)

[11670547439198173018840584368947199999]


In [25]:
len(new_primes1)

1

this sequential way of taking two numbers and replacing it with third number which is x+y+x*y gives a very large number. Q is an insanely large number and is definitely not the lowest achievable. let us try anoher approach.let us combine first and last of the list at each iteration and check what we get again.

In [26]:
new_primes2 = primes.copy()
while(1):
  if len(new_primes2) == 1:
    break
  a = new_primes2[0]
  b = new_primes2[-1]
  c = a + b + a*b # also a prime
  new_primes2.pop(0)
  new_primes2.pop(-1)
  new_primes2.insert(-1,c)

print(new_primes2)

[11670547439198173018840584368947199999]


In [27]:
new_primes1[0] - new_primes2[0]

0

In [30]:
import time

Both the approaches give the same results. the final Q value is the same.

In [50]:
def sieve_of_eratosthenes(max_num):
    """Return a list of all prime numbers up to max_num."""
    is_prime = [True] * (max_num + 1)
    is_prime[0] = is_prime[1] = False

    for start in range(2, int(max_num**0.5) + 1):
        if is_prime[start]:
            for multiple in range(start*start, max_num + 1, start):
                is_prime[multiple] = False

    return [num for num, prime in enumerate(is_prime) if prime]

def smallest_final_value(max_num):
    # Get all primes between 1 and max_num
    primes = sieve_of_eratosthenes(max_num)

    # Compute the product of (p_i + 1) for each prime p_i
    product = 1
    for prime in primes:
        product *= (prime + 1)

    # The result is this product minus 1
    return product - 1

# Define the range
max_number = 1000

# Find the smallest final value
start_time = time.time()
result = smallest_final_value(max_number)
execution_time = time.time() - start_time

print("execution time without parallelization : ", execution_time)

# Print the result
print(f"The smallest value of Q is: {result}")


execution time without parallelization :  0.0020780563354492188
The smallest value of Q is: 147112613210319486499676296015469376140521397487772312539818224915030479615918413181667804881965140978577602879012396476181147114517095334951755284774296929590260545569265720366451220763460675369225562092691997648611896298415668747092796050814927021662304746378459453311470775199933251758665556263651853490184370195055575968818253407088121287627256915970363971231352356863999999999999999999999999999999999999999999999


In [19]:
result - new_primes1[0]

0

No matter which two numbers you take and do this continuously, you get the same value of Q. At the end, x+y+x.y = (x+1). (y+1) - 1. So, we need to find the product of all primes + 1 and subtract from 1 in the end.

Commutativity : x+y+x.y
if we replace x with y and y with x -> y+x+y.x which is same as original expression, meaning the operation is commutative. \
Associativity : \
(x+y+x.y) + z + (x+y+x.y)*z = x + y + z + x.y + z.x + z.y + x.y.z , if we now combine y and z first and then do this operation with x , we get : \
y+z+y.z + x + (y+z+y.z)*x = y+z+x+y.z+x.y+x.z+x.y.z ,which is same as above expression hence it is associate, thus this concludes that the order in which we do this till we are left with one number does not matter, it is going to reach the same result. \
We need to add 1 to every prime number and compute the product. \
This can be parallelized. \
for a single processor, first we add one to every number - n operations. \
then we iterate through every element and keep computing products , so it is still n - 1 multiplications. finally, we subtract 1 from the answer. \
In case of parallelization: \
we have 168 numbers, let us say we have 8 cores, we shift 84 numbers to core 2, and then half of the existing inputs to core 3 and 4 and so on , till we have number equally distributed among 8 cores, 21 numbers per core. \
then we can add 1 to each number parallely using vectorization, load four numbers into the vector , add 1 and then load it back to memory. \
product can be computed per core and the product can be saved at the last location. once we have products for all 8 cores, we then do prefix product like we did for sum before in log2 times. finally we can take the product and subtract from 1 to get the result.


here is the python implementation of the above idea of parallelization using threads. [chatgpt helped me with the implementation to be honest!]

In [48]:
import concurrent.futures
from functools import reduce
from operator import mul

def sieve_of_eratosthenes(max_num):
    """Return a list of all prime numbers up to max_num."""
    is_prime = [True] * (max_num + 1)
    is_prime[0] = is_prime[1] = False

    for start in range(2, int(max_num**0.5) + 1):
        if is_prime[start]:
            for multiple in range(start*start, max_num + 1, start):
                is_prime[multiple] = False

    return [num for num, prime in enumerate(is_prime) if prime]

def parallel_product(numbers):
    """Compute the product of a list of numbers in parallel."""
    def worker(chunk):
        return reduce(mul, chunk, 1)

    num_threads = min(len(numbers), 2)  # Number of parallel workers (adjust based on your system)
    chunk_size = len(numbers) // num_threads
    chunks = [numbers[i:i + chunk_size] for i in range(0, len(numbers), chunk_size)]

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return reduce(mul, results, 1)

def smallest_final_value(max_num):
    # Get all primes between 1 and max_num
    primes = sieve_of_eratosthenes(max_num)

    # Compute the product of (p_i + 1) for each prime p_i in parallel
    transformed_numbers = [p + 1 for p in primes]
    product = parallel_product(transformed_numbers)

    # The result is this product minus 1
    return product - 1

# Define the range
max_number = 1000

# Find the smallest final value
start_time= time.time()
result = smallest_final_value(max_number)
execution_time = time.time() - start_time

print("execution time with parallelization : ", execution_time)

# Print the result
print(f"The smallest value of Q is: {result}")


execution time with parallelization :  0.0008478164672851562
The smallest value of Q is: 147112613210319486499676296015469376140521397487772312539818224915030479615918413181667804881965140978577602879012396476181147114517095334951755284774296929590260545569265720366451220763460675369225562092691997648611896298415668747092796050814927021662304746378459453311470775199933251758665556263651853490184370195055575968818253407088121287627256915970363971231352356863999999999999999999999999999999999999999999999
